# Proyecto 2
## Data Science, sección 40
## Grupo 1
Javier Alejandro Ovalle Chiquín, 22103  
José Ángel Morales Farfan, 22689  
Ricardo Josué Morales Contreras, 22289  
Karen Daniela Pineda Ventura 231132

## Planteamiento inicial del problema

### Situación problemática
Los estudiantes realizan resúmenes en respuesta a una consigna, luego evaluadores puntúan cada resumen en dos factores: content (Si el resumen cubre lo solicitado) y wording (calidad de la redacción). Evaluar manualmente esto es costoso, lento y potencialmente inconsistente. Por lo que, automatizar la evaluación con modelos de PLN puede permitir retroalimentación a escala, detectar rápido resúmenes que necesiten intervención y homogeneizar criterios.

### Problema científico
Construir modelos de Procesamiento del Lenguaje Natural que, a partir del texto del resumen y del enunciado del prompt, predigan dos puntuaciones continuas (content y wording) de forma que se reduzca la discrepancia con respecto a las evaluaciones realizadas manualmente.

### Objetivos

Objetivo general

- Diseñar, implementar y evaluar un pipeline de PLN que prediga las puntuaciones content y wording de resúmenes estudiantiles con una evaluación cuantitativa reproducible (RMSE, correlaciones).

Objetivos específicos

- Implementar preprocesamiento reproducible (tokenización, limpieza) y documentar su efecto en la calidad de features

- Entrenar un baseline (TF-IDF + Ridge) validado por prompt con GroupKFold y obtener RMSE por target; comparar resultados con una línea base (por ejemplo, promedio)

- Realizar EDA que identifique valores faltantes, distribución de scores, outliers y diferencias entre prompts


## Descripción de los datos

Variables principales
- student_summary: texto escrito por estudiantes
- prompt: enunciado al que responde el resumen
- content: puntuación otorgada al contenido
- wording: puntuación de la redacción

Operaciones de limpieza
- Conversión a minúsculas
- Eliminación de signos de puntuación, caracteres especiales y stopwords
- Tokenización y lematización
- Manejo de valores faltantes (textos vacíos se reemplazan por cadenas vacías)

## Investigación preliminar

**Técnicas comunes para detección de patrones de texto**

**El preprocesamiento de Lenguaje Natural (PLN)** busca extraer conocimiento útil a partir de textos. En problemas como el de la competencia, el objetivo es detectar patrones en los resúmenes escritos por estudiantes que permitan estimar automáticamente la calidad de su contenido y redacción.

Entre las técnicas están:

Preprocesamiento de texto
Antes de aplicar cualquier modelo, es necesario transformar el texto original para que sea más manejable:
- Normalización: pasar todo a minúsculas, eliminar signos de puntuación, URLs y caracteres especiales. Esto disminuye el ruido y hace más consistentes los datos.
- Tokenización: dividir el texto en unidades (tokens), que suelen ser palabras o subpalabras. Ejemplo: "The boy runs fast" = ["the", "boy", "runs", "fast"].
- Stopwords  removal: eliminación de palabras vacías (the, and, is), que no aportan tanta información
- Lematización/Stemming: reducir palabras a su forma base ("running" = "run"). Permite agrupar variaciones morfológicas
- Correción de valores faltantes y outliers: textos muy cortos, vacíos o muy alrgos deben analizarse aparte

**Representación vectorial del texto**

El texto debe representarse numéricamente para que un modelo pueda procesarlo.

**Bolsa de palabras (Bag of words):** Cada documento es representado como un vector de frecuencias de palabras, ignorando el orden. Es simple y útil para detectar patrones de vocabulario físico.

**TF-IDF (Term Frecuency - Inverse Document Frecuency):**
Pondera palabras según su frecuencia en un documento, penaliza las que son muy frecuentes, lo cual permite detectar palabras clave más relevantes para diferenciar textos.

**N-gramas:** Considera secuencias de palabras (bigramas, trigramas). Esto captura patrones de estilo y frases típicas de resúmenes.

**Embeddings semánticos:**
- Word2Vec, Glove, FastText, representan palabras en vectores densos que capturan similitud semántica ("dog" y "puppy").
- Embeddings contextuales (Transformers como BERT, RoBERTa, DistilBERT) capturan el significado de cada palabra en función de su contexto, lo que permite representar mejor la semántica de frases completas.

Patrones estilísticos y estructurales
Se pueden usar métricas como:
- Longitud del texto: cantidad de caracteres o palabras
- Riqueza léxica: proporción de palabras únicas frente al total
- Medidas de legibilidad: índices como Flesh-Kincaid, que miden la complejidad sintáctica
- Distribución de puntuación y longitud de oraciones: redacción más clara suele usar estructuras más balanceadas

**Modelos para detección de patrones**

Regresión lineal y Ridge/Lasso: adecuados como modelos base para datos vectorizados (TF-IDF).

Árboles de decisión y ensambles (Random Forest, XGBoost, LightGBM): capturan patrones no lineales en los features.

Redes neuronales: CNN y RNN, detectan secuencias recurrentes en textos.

Transformers (BERT, RoBERTa, GPT, etc): de última generación, capaces de capturar relaciones complejas y contexto profundo, especialmente efectivos evaluación de calidad de texto.

**Evaluación de patrones**

Métricas cuantitativas: RMSE (Root Mean Squared Error), MAE (Mean Absolute Error) y correlaciones estadísticas.

Validación cruzada estratificada por prompt: asegura que el modelo no se sobreajuste, ya que los resúmenes son de temas distintos.



Ahora cargaremos las librerías necesarias para el proyecto

In [1]:
import os
from pathlib import Path
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error

from sklearn.utils import indexable
from sklearn.model_selection import check_cv

Para la configuración:

In [2]:
# Visual config
plt.style.use('seaborn-v0_8-deep')
plt.rcParams['figure.figsize'] = (8,5)

# Inicializar NLTK
for resource in ['punkt', 'punkt_tab', 'stopwords']:
    try:
        nltk.data.find(resource)
    except LookupError:
        nltk.download(resource)

STOPWORDS = set(stopwords.words('english'))

# Directorio de datos
DATA_DIR = Path('.')

[nltk_data] Downloading package punkt to C:\Users\Javier
[nltk_data]     Chiquin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Javier
[nltk_data]     Chiquin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Javier
[nltk_data]     Chiquin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
